# Simulations: profiling and performance
## Random walks

In this notebook, we will look at one of the simplest types of Monte Carlo numerical simulation, random walks.

In the simplest random walk, a particle starts at $0$ and jumps to the left ($-1$) or the right ($+1$) with equal probability.

The following is a simple implementation of a single random walk:

In [ ]:
@time begin
    
    numsteps = 1000
    pos = 0 
    for j in 1:numsteps

        if rand() < 0.5
            step = -1
        else
            step = +1
        end

        pos += step 
    end
    
end

Let's wrap it in a function, which is good programming practice, and allows us to have `numsteps` as a paramater.
It turns out to have an additional, important effect in Julia.

In [ ]:
"""Single 1D random walk from the origin.
Returns the final position after `numsteps` steps."""
function walk(numsteps=1000)  # default value of the parameter
    
    pos = 0 
    
    for j in 1:numsteps

        if rand() < 0.5   # can replace by rand(Bool)
            step = -1
        else
            step = +1
        end

        pos += step 
    end
    
    return pos
    
end

And then, let us check if it works. 

In [ ]:
@time walk(1)

In [ ]:
@time walk(100)

In [ ]:
@time walk(1000)

## Draw a random walk

One way to understand what each walker is doing is by visualizing its path. In Julia, most visualization is done via the `Plots` package, which an umbrella package with a uniform API across different plotting libraries (aka *backends*). Let's load the `plotly` backend.

In [ ]:
using Plots; plotly()

Let us record the position at each step via a new function `trajectory`

In [ ]:
function trajectory(numsteps=1000)

    pos = 0 
    positions = [pos]

    for j in 1:numsteps

        if rand() < 0.5
            step = -1
        else
            step = +1
        end

        pos += step 
        push!(positions, pos)

    end
    
    positions
end


And now, plot:

In [ ]:
numsteps = 1000
plot(1:numsteps, trajectory(numsteps))

Now let us get a sense of how much time this takes. 

In [ ]:
using Interact

In [ ]:
@manipulate for k in 3:9
    @elapsed walk(10^k)
end

In [ ]:
plot(3:9, [@elapsed walk(10^k) for k = 3:9])

## Add parallelism

Now that we have a sense of how much time it takes, we now offload work to other Julia processes. First, let's retrieve 8 Julia processes. 

In [ ]:
using JuliaRunClient
ctx = Context()
nb = self()

In [ ]:
initParallel()
@result setJobScale(ctx, nb, 3)
waitForWorkers(3)

In [ ]:
# addprocs(8) if you're on your laptop 

Let's now use `DArrays` to parallelize this random walk. 

In [ ]:
@everywhere using DistributedArrays

Here's our `walk` function again. 

In [ ]:
@everywhere function walk(numsteps)
    pos = 0

    for j in 1:numsteps
        
        if rand(Bool)  # NB
            step = -1
        else
            step = +1
        end
        
        pos += step # ifelse(rand() < 0.5, -1, +1)
    end
    
    return pos
end

Let us define how many walkers we want and how many steps we want them to walk. In serial, all our walkers are present on a single process.

In [ ]:
@everywhere begin
    numsteps   = 10000
    numwalkers = 100000 
end
serialwalkers = collect(1:numwalkers)


But with `distribute`, the walkers are distributed across all worker processes.

In [ ]:
parallelwalkers = distribute(serialwalkers)

And, as earlier, we can examine the distribution by looking at the indices stored on each worker.

In [ ]:
parallelwalkers.indexes

In [ ]:
typeof(parallelwalkers)

### Benchmarking

Most benchmarking in Julia is done via the package `BenchmarkTools`. 

In [ ]:
using Compat
using BenchmarkTools

Let us perform the random walks by calling the `map` function on all the workers. 

In serial:

In [ ]:
@benchmark map(_ -> walk(numsteps), serialwalkers)

In parallel:

In [ ]:
@benchmark positions = map( _ -> walk(numsteps), parallelwalkers)